In [7]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import IPython.display as display
import socket
from pandas import DataFrame
from hsa import emu_connect
import importlib
from ipywidgets import widgets

In [ ]:
# hack to import modules in pycharm and browser
import sys
sys.path.append("../")

In [8]:
%matplotlib inline

In [9]:
prim_soc = socket.create_connection(("localhost", 9090))
prim_soc

<socket.socket fd=38, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 54272), raddr=('127.0.0.1', 9090)>

In [10]:
importlib.reload(emu_connect)
emu = emu_connect.Emu2(prim_soc)

In [6]:
emu.close()

'true\n'

In [8]:
emu.step()

'true\n'

In [11]:
basic_ram = emu.get_ram()

In [35]:
emu.play_beginning()

'true\n'

In [ ]:
emu.speed_mode("maximum")

In [22]:
emu.play_movie()

True

In [ ]:
def visualize_ram(ram_bytes):
    return plt.imshow(np.frombuffer(ram_bytes, dtype=np.dtype(np.uint8)).reshape((32,64))
               ,vmin=0,vmax=255,interpolation="nearest",cmap=plt.get_cmap('binary'))

In [ ]:
for _ in range(4):
    for _ in range(60):
        emu.step()
    visualize_ram(emu.get_ram())
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [ ]:
def ram_to_numpy(ram_bytes):
    return np.frombuffer(ram_bytes, dtype=np.dtype(np.uint8)).reshape((32, 64))

def next_nram():
    for _ in range(60):
        emu.step()
    return ram_to_numpy(emu.get_ram())

fig = plt.figure()
plt.imshow(ram_to_numpy(emu.get_ram()), vmin=0, vmax=255, interpolation="nearest", cmap=plt.get_cmap('binary'))
ani = animation.FuncAnimation(fig, next_nram, interval=40, blit=True)

In [ ]:
plt.ion()
for _ in range(4):
    for _ in range(60):
        emu.step()
    
    visualize_ram(emu.get_ram())
    plt.draw()
    # display.clear_output(wait=True)
    # display.display(plt.gcf())

In [17]:
for i in range(240):
    emu.step()

In [ ]:
# clear recv cache

In [14]:
def mario_x_from_ram(ram):
    return ram[0x86]

In [15]:
def play_movie(emu:emu_connect.Emu2):
    # sanity hack, to make schour nothing is left in the buffer
    # prim_soc.recv(2**10)
    emu.play_beginning()
    emu.speed_mode("turbo")
    last_state = True
    while last_state:
        last_state = emu.play_movie()
        yield emu.get_ram()

In [18]:
for frame in play_movie(emu):
    mario_x_from_ram(frame)

In [ ]:
tex = widgets.Text(value='Hello World!', disabled=True) 
display.display(tex)
for frame in play_movie(emu):
    tex.value = str(mario_x_from_ram(frame))

In [49]:
first_recording = DataFrame.from_records(play_movie(emu))

In [ ]:
first_recording